# Single-Step Tool Use

- **여러 도구:** 예제에는 도구가 하나만 있었지만, 모델이 여러 도구를 사용할 수 있다면 어떻게 될까? 모델은 특정 시점에 어떤 도구를 사용할지 어떻게 추론하는가?
- **병렬 도구 호출:** 우리는 단일 단계에서 단일 도구 호출만 보았지만, 둘 이상의 도구 호출이 필요한 경우 어떻게 보일까?
- **도구를 사용하지 않을 때:** 질문이 도구 없이 모델에 의해 직접 답변될 수 있고, 그렇게 해야 한다면 어떻게 되는가?
- **채팅 환경에서의 도구 사용:** 사용자가 후속 질문을 하는 경우, 즉 어시스턴트가 다중 턴 채팅 환경에서 대화의 맥락을 어떻게 유지하는가?

## 도구 설정하기

먼저 OpenAI 클라이언트를 설정하자. 나중에 사용할 `json` 모듈도 가져와야 한다.

In [1]:
import os
import json
from typing import List, Dict
from openai import OpenAI
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

### 1단계: 도구 생성하기

이제 두 가지 도구를 만들어 보자:

- 모의 판매 데이터베이스를 포함하는 판매 데이터베이스 쿼리 함수 `daily_sales_report` (이전 장과 동일)
- 모의 제품 카탈로그를 포함하는 제품 카탈로그 쿼리 함수 `product_database` (새로운 함수)

In [2]:
def daily_sales_report(day: str) -> str:
    """
    주어진 날짜의 판매 보고서를 검색하는 함수다.
    """
    sales_database = {
        '2025-08-28': {'total_sales_amount': 1000000, 'total_units_sold': 100},
        '2025-08-29': {'total_sales_amount': 15000000, 'total_units_sold': 150},
        '2025-08-30': {'total_sales_amount': 8000000, 'total_units_sold': 80}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        result = {
            'date': day,
            'summary': f"총 판매 금액: {report['total_sales_amount']}, 총 판매 수량: {report['total_units_sold']}"
        }
    else:
        result = {'date': day, 'summary': '해당 날짜에 대한 판매 데이터가 없습니다.'}
    
    return json.dumps(result, ensure_ascii=False)

In [3]:
def product_database(category: str) -> str:
    """
    주어진 카테고리의 제품을 검색하는 함수다.
    """
    product_catalog = {
        'Electronics': [
            {'product_id': 'E1001', 'name': 'Smartphone', 'price': 100000, 'stock_level': 20},
            {'product_id': 'E1002', 'name': 'Laptop', 'price': 200000, 'stock_level': 15},
            {'product_id': 'E1003', 'name': 'Tablet', 'price': 50000, 'stock_level': 25},
        ],
        'Clothing': [
            {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 2000, 'stock_level': 100},
            {'product_id': 'C1002', 'name': 'Jeans', 'price': 5000, 'stock_level': 80},
            {'product_id': 'C1003', 'name': 'Jacket', 'price': 10000, 'stock_level': 40},
        ]
    }
    
    products = product_catalog.get(category, [])
    result = {
        'category': category,
        'products': products
    }
    return json.dumps(result, ensure_ascii=False)

available_tools = {
    "daily_sales_report": daily_sales_report,
    "product_database": product_database
}

### 2단계: 도구 스키마 정의하기

다음으로, 두 도구에 대한 스키마를 정의한다. 두 함수는 각각 `day`와 `category`라는 하나의 매개변수를 받는다.

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "daily_sales_report",
            "description": "데이터베이스에 연결하여 특정 날짜의 전체 판매량 및 판매 정보를 검색한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "description": "YYYY-MM-DD 형식으로 된, 판매 데이터를 검색할 날짜.",
                    }
                },
                "required": ["day"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "product_database",
            "description": "카테고리, 가격, 재고 수준 등 이 회사의 모든 제품에 대한 정보를 포함하는 데이터베이스다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "이 카테고리의 모든 제품에 대한 제품 정보를 검색한다.",
                    }
                },
                "required": ["category"]
            }
        }
    }
]

### 3단계: 시스템 프롬프트 정의하기 (선택 사항)

이전 장과 동일한 시스템 프롬프트를 유지한다.

In [5]:
system_prompt = """당신은 사용자의 질문과 요청에 대화형으로 답변하는 AI 어시스턴트입니다. 
다양한 주제에 대한 광범위한 요청을 받게 될 것입니다. 
답변을 조사하는 데 도움이 되는 다양한 검색 엔진이나 유사한 도구를 갖추고 있습니다. 
사용자의 요구를 최선을 다해 지원하는 데 집중해야 합니다.
사용자가 다른 스타일의 답변을 요청하지 않는 한, 완전한 문장으로 올정한 문법과 철자법을 사용하여 답변해야 합니다.
"""

## 단일 단계 도구 사용

이제 이전 장에서 사용한 코드를 가져와 네 단계의 도구 사용 워크플로우를 자동화하는 `run_assistant` 함수를 만들어 보자. 이 함수는 다음을 수행한다:

- 사용자 메시지를 받고 대화 기록에 추가한다 (1단계)
- Chat API를 호출하여 도구 호출을 생성한다 (2단계)
- 응답에 도구 호출이 하나 이상 포함된 경우, 도구 호출을 실행하고 결과를 얻는다 (3단계)
- 도구 결과를 포함하여 다시 Chat API를 호출하여 최종 응답을 생성한다 (4단계)

In [6]:
MODEL = "gpt-4o-mini"

def run_assistant(message: str, chat_history: List[Dict] = None) -> List[Dict]:
    # 1단계: 사용자 메시지 받기
    print(f"질문:\n{message}")
    print("="*50)
    
    if chat_history is None:
        chat_history = [{"role": "system", "content": system_prompt}]
    
    # 현재 사용자 메시지를 대화 기록에 추가
    chat_history.append({"role": "user", "content": message})
    
    # 2단계: 도구 호출 생성 (필요한 경우)
    response = client.chat.completions.create(
        model=MODEL,
        messages=chat_history,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    chat_history.append(response_message) # 어시스턴트의 응답(도구 호출 포함)을 기록에 추가

    tool_calls = response_message.tool_calls

    # 도구 호출이 필요한 경우에만 3, 4단계 실행
    if tool_calls:
        print("도구 호출:")
        for call in tool_calls:
            print(f"- 도구: {call.function.name} | 매개변수: {call.function.arguments}")
        print("="*50)
        
        # 3단계: 도구 결과 얻기
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_tools[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            
            # 도구 실행 결과를 대화 기록에 추가
            chat_history.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        
        # 4단계: 최종 응답 생성
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=chat_history
        )
        final_text = final_response.choices[0].message.content
        chat_history.append(final_response.choices[0].message) # 최종 응답을 기록에 추가
    else:
        # 도구 호출이 필요 없는 경우, 첫 번째 응답이 최종 응답이다.
        final_text = response_message.content

    # 최종 응답 출력
    print("최종 응답:")
    print(final_text)
    print("="*50)

    return chat_history

In [7]:
chat_history = run_assistant("2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?")

질문:
2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?
도구 호출:
- 도구: daily_sales_report | 매개변수: {"day":"2025-08-29"}
최종 응답:
2025년 8월 29일의 판매 요약은 다음과 같습니다. 총 판매 금액은 15,000,000원이었으며, 총 판매 수량은 150개였습니다. 더 궁금한 사항이 있으시면 말씀해 주세요.


어시스턴트는 사용 가능한 두 도구 중에서 `daily_sales_report`가 질문에 답하기에 충분하다고 정확하게 식별하고, 질문에 올바르게 답변한다.

이 예제에서 모델은 질문에 답하기 위해 단일 단계가 필요하며, 그 단일 단계 내에서 단 하나의 도구 호출만 필요하다.

## 단일 단계, 병렬 도구 사용

앞서 언급했듯이, 모델은 응답을 제공하기 위해 둘 이상의 도구가 필요하다고 결정할 수 있다. 이는 동일한 단계 내에서 여러 도구를 병렬로 호출하는 것을 의미한다. 이는 다음과 같을 수 있다:

- 다른 도구를 병렬로 호출하기
- 동일한 도구를 여러 번 병렬로 호출하기
- 또는 둘의 조합

In [8]:
chat_history_parallel = run_assistant(
    "2025년 8월 28일과 29일의 판매 요약과 'Electronics' 카테고리 제품의 재고 수준을 알려줄 수 있나요?"
)

질문:
2025년 8월 28일과 29일의 판매 요약과 'Electronics' 카테고리 제품의 재고 수준을 알려줄 수 있나요?
도구 호출:
- 도구: daily_sales_report | 매개변수: {"day": "2025-08-28"}
- 도구: daily_sales_report | 매개변수: {"day": "2025-08-29"}
- 도구: product_database | 매개변수: {"category": "Electronics"}
최종 응답:
2025년 8월 28일과 29일의 판매 요약은 다음과 같습니다:

- **2025년 8월 28일**: 
  - 총 판매액: 1,000,000 원
  - 판매 건수: 100 건

- **2025년 8월 29일**: 
  - 총 판매액: 15,000,000 원
  - 판매 건수: 150 건

또한, 'Electronics' 카테고리 제품의 재고 수준은 다음과 같습니다:

1. **스마트폰 (Smartphone)**
   - 가격: 100,000 원
   - 재고 수준: 20 개

2. **노트북 (Laptop)**
   - 가격: 200,000 원
   - 재고 수준: 15 개

3. **태블릿 (Tablet)**
   - 가격: 50,000 원
   - 재고 수준: 25 개

더 궁금한 점이나 필요하신 정보가 있다면 말씀해 주세요!


## 직접 답변하기

In [9]:
chat_history_direct1 = run_assistant(
    "성장하는 회사를 만드는 방법에 대한 간결한 팁 3가지를 알려주세요."
)

질문:
성장하는 회사를 만드는 방법에 대한 간결한 팁 3가지를 알려주세요.
최종 응답:
성장하는 회사를 만드는 방법에 대한 간결한 팁 3가지는 다음과 같습니다:

1. **명확한 비전과 목표 설정**: 회사의 방향성과 목표를 명확히 하고, 이를 바탕으로 전략을 수립하세요. 직원들과의 일관된 소통을 통해 모두가 같은 목표를 향해 나아가도록 합니다.

2. **고객 중심의 접근**: 고객의 피드백을 적극적으로 수용하고, 그들의 필요와 요구에 맞춘 제품이나 서비스를 제공하세요. 고객 만족도와 충성도를 높이는 것이 지속 가능한 성장에 기여합니다.

3. **디지털화 및 혁신 촉진**: 최신 기술을 활용하여 업무 효율성을 증대시키고, 시장의 변화에 빠르게 대응하세요. 혁신적인 아이디어와 프로세스를 통해 경쟁력을 유지하는 것이 중요합니다.


In [10]:
chat_history_direct2 = run_assistant("현재 회사의 직원 수는 몇 명인가요?")

질문:
현재 회사의 직원 수는 몇 명인가요?
최종 응답:
죄송하지만, 현재 회사의 직원 수에 대한 정보는 가지고 있지 않습니다. 해당 정보를 확인하려면 인사 부서나 관련 부서에 문의하셔야 할 것 같습니다. 다른 질문이나 요청이 있으시면 도와드리겠습니다!


하지만 사용 가능한 도구 중 어느 것도 이 정보를 제공할 수 없기 때문에 모델은 어떤 도구 호출도 시도하지 않는다. 대신 사용자 질문에 직접 응답하여 질문에 답하는 데 필요한 정보가 없다고 언급한다.

## 상태 관리 (메모리)

OpenAI API에서 각 턴의 채팅 기록은 다음 메시지들의 순차적인 목록이다:

- `user` 역할의 메시지
- `assistant` 역할의 메시지 (도구 호출 목록 포함)
- `tool` 역할의 메시지 (도구 결과 목록 포함)
- 마지막으로 `assistant` 역할의 메시지 (사용자에 대한 최종 응답)

In [11]:
# 첫 번째 턴
chat_history = run_assistant("2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?")

질문:
2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?
도구 호출:
- 도구: daily_sales_report | 매개변수: {"day":"2025-08-29"}
최종 응답:
2025년 8월 29일의 판매 요약은 다음과 같습니다. 총 판매 금액은 15,000,000원이었으며, 총 판매 수량은 150개였습니다.


이전과 동일한 답변을 제공한다.

이제 후속 질문을 해보자. 이전 턴의 맥락 없이는 모델이 답변할 수 없는 다소 모호한 질문이다. 여기서 첫 번째 턴의 `chat_history`를 `run_assistant` 함수에 전달한다.

In [12]:
# 두 번째 턴
chat_history = run_assistant("28일은 어땠나요?", chat_history)

질문:
28일은 어땠나요?
도구 호출:
- 도구: daily_sales_report | 매개변수: {"day":"2025-08-28"}
최종 응답:
2025년 8월 28일의 판매 요약은 다음과 같습니다. 총 판매 금액은 1,000,000원이었으며, 총 판매 수량은 100개였습니다.


모델은 이전 턴에서 물었던 내용 때문에 "28일"이 2025년 8월 28일을 의미한다고 추론할 수 있다.

대화를 계속해 보자.

In [13]:
# 세 번째 턴
chat_history = run_assistant("이틀 동안 팔린 총 수량은 몇 개인가요?", chat_history)

질문:
이틀 동안 팔린 총 수량은 몇 개인가요?
최종 응답:
2025년 8월 28일과 29일 동안 팔린 총 수량은 다음과 같습니다.

- 8월 28일: 100개
- 8월 29일: 150개

따라서 이틀 동안 팔린 총 수량은 100개 + 150개 = 250개입니다.


다시 한번, 모델은 채팅 맥락에서 "이틀"이 무엇을 의미하는지 추론할 수 있다.

채팅 기록을 살펴보자. 이는 `user`, `assistant`, `tool` 메시지들이 올바른 턴 순서로 추가된 집합으로 구성되어, 모델이 새로운 응답을 생성할 때마다 올바른 맥락을 제공한다.

In [14]:
import pprint

pprint.pprint(chat_history)

[{'content': '당신은 사용자의 질문과 요청에 대화형으로 답변하는 AI 어시스턴트입니다. \n'
             '다양한 주제에 대한 광범위한 요청을 받게 될 것입니다. \n'
             '답변을 조사하는 데 도움이 되는 다양한 검색 엔진이나 유사한 도구를 갖추고 있습니다. \n'
             '사용자의 요구를 최선을 다해 지원하는 데 집중해야 합니다.\n'
             '사용자가 다른 스타일의 답변을 요청하지 않는 한, 완전한 문장으로 올정한 문법과 철자법을 사용하여 답변해야 '
             '합니다.\n',
  'role': 'system'},
 {'content': '2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Htcrye504qi1Va8tdKHWT401', function=Function(arguments='{"day":"2025-08-29"}', name='daily_sales_report'), type='function')]),
 {'content': '{"date": "2025-08-29", "summary": "총 판매 금액: 15000000, 총 판매 수량: '
             '150"}',
  'name': 'daily_sales_report',
  'role': 'tool',
  'tool_call_id': 'call_Htcrye504qi1Va8tdKHWT401'},
 ChatCompletionMessage(content='2025년 8월 29일의 판매 요약은 다음과 같습니다. 총 판매 금액은 15,000,000원이었으며, 총 